In [1]:
!pip install --upgrade pyiceberg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 1.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 3.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 5.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 2.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Su

In [2]:
!pip install pyiceberg[s3fs,sql-sqlite]

In [3]:
!pip install --upgrade pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 3.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 15.0.2
    Uninstalling pyarrow-15.0.2:
      Successfully uninstalled pyarrow-15.0.2


In [4]:
!pip install pandas

In [5]:
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.fs as fs

In [6]:
# Crear bucket manualmente antes (usando boto3 o minio client)
#import boto3
#s3 = boto3.client('s3', 
                 #endpoint_url='http://minio:9000',
                 #aws_access_key_id='admin',
                 #aws_secret_access_key='password')
#s3.create_bucket(Bucket='new-bucket')

In [7]:
minio = fs.S3FileSystem(
    access_key="admin",
    secret_key="password",
    endpoint_override="http://minio:9000",
)

In [8]:
path = "warehouse/yellow_trip_20250912031341/yellow_trip_data"

In [9]:
dataset_3 = ds.dataset(path, filesystem=minio, format="parquet")

In [10]:
table_3 = dataset_3.to_table()
print(table_3.schema)

vendor_id: int32
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: int64
trip_distance: double
ratecode_id: int64
store_and_fwd_flag: large_string
pu_location_id: int32
do_location_id: int32
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
cbd_congestion_fee: double


In [11]:
for file_info in minio.get_file_info([path, "taxis/taxis_parquet/df_data"]):
    print(file_info)


<FileInfo for 'warehouse/yellow_trip_20250912031341/yellow_trip_data': type=FileType.Directory>
<FileInfo for 'taxis/taxis_parquet/df_data': type=FileType.NotFound>


In [12]:
arrow_schema = table_3.schema
print(arrow_schema)

vendor_id: int32
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: int64
trip_distance: double
ratecode_id: int64
store_and_fwd_flag: large_string
pu_location_id: int32
do_location_id: int32
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
cbd_congestion_fee: double


In [13]:
from pyiceberg.catalog import load_catalog

catalog = load_catalog(
    "nessie",
    **{
        "uri": "http://nessie:19120/iceberg/main/",
    },
)


In [14]:
print(catalog.properties)

{'rest-metrics-reporting-enabled': 'false', 'warehouse': 's3://my-bucket', 'client.region': 'us-east-1', 'io-impl': 'org.apache.iceberg.aws.s3.S3FileIO', 'rest-page-size': '200', 'prefix': 'main', 'uri': 'http://nessie:19120/iceberg/', 's3.endpoint': 'http://minio:9000/', 's3.path-style-access': 'true', 'nessie.core-base-uri': 'http://nessie:19120/api/', 'nessie.catalog-base-uri': 'http://nessie:19120/catalog/v1/', 'nessie.iceberg-base-uri': 'http://nessie:19120/iceberg/', 'nessie.is-nessie-catalog': 'true', 'nessie.prefix-pattern': '{ref}|{warehouse}', 'nessie.default-branch.name': 'main'}


In [15]:
namespaces = catalog.list_namespaces()
print(namespaces)


[]


In [16]:
catalog.create_namespace("taxis")

namespaces = catalog.list_namespaces()
print(namespaces)

[('taxis',)]


In [17]:
catalog.create_table(
    "taxis.taxis_iceberg", 
    arrow_schema,
)

taxis_iceberg(
  1: vendor_id: optional int,
  2: tpep_pickup_datetime: optional timestamp,
  3: tpep_dropoff_datetime: optional timestamp,
  4: passenger_count: optional long,
  5: trip_distance: optional double,
  6: ratecode_id: optional long,
  7: store_and_fwd_flag: optional string,
  8: pu_location_id: optional int,
  9: do_location_id: optional int,
  10: payment_type: optional long,
  11: fare_amount: optional double,
  12: extra: optional double,
  13: mta_tax: optional double,
  14: tip_amount: optional double,
  15: tolls_amount: optional double,
  16: improvement_surcharge: optional double,
  17: total_amount: optional double,
  18: congestion_surcharge: optional double,
  19: airport_fee: optional double,
  20: cbd_congestion_fee: optional double
),
partition by: [],
sort order: [],
snapshot: null

In [18]:
taxis_iceberg = catalog.load_table("taxis.taxis_iceberg")
taxis_iceberg.append(table_3)

In [19]:
# Después de crear la tabla, verifica dónde quedó
#taxis_iceberg = catalog.load_table("taxis.taxis_iceberg")
print(f"Ubicación real: {taxis_iceberg.location}")

Ubicación real: <bound method Table.location of taxis_iceberg(
  1: vendor_id: optional int,
  2: tpep_pickup_datetime: optional timestamp,
  3: tpep_dropoff_datetime: optional timestamp,
  4: passenger_count: optional long,
  5: trip_distance: optional double,
  6: ratecode_id: optional long,
  7: store_and_fwd_flag: optional string,
  8: pu_location_id: optional int,
  9: do_location_id: optional int,
  10: payment_type: optional long,
  11: fare_amount: optional double,
  12: extra: optional double,
  13: mta_tax: optional double,
  14: tip_amount: optional double,
  15: tolls_amount: optional double,
  16: improvement_surcharge: optional double,
  17: total_amount: optional double,
  18: congestion_surcharge: optional double,
  19: airport_fee: optional double,
  20: cbd_congestion_fee: optional double
),
partition by: [],
sort order: [],
snapshot: Operation.APPEND: id=6019170165029696754, schema_id=0>


In [20]:
print(catalog.properties)

{'rest-metrics-reporting-enabled': 'false', 'warehouse': 's3://my-bucket', 'client.region': 'us-east-1', 'io-impl': 'org.apache.iceberg.aws.s3.S3FileIO', 'rest-page-size': '200', 'prefix': 'main', 'uri': 'http://nessie:19120/iceberg/', 's3.endpoint': 'http://minio:9000/', 's3.path-style-access': 'true', 'nessie.core-base-uri': 'http://nessie:19120/api/', 'nessie.catalog-base-uri': 'http://nessie:19120/catalog/v1/', 'nessie.iceberg-base-uri': 'http://nessie:19120/iceberg/', 'nessie.is-nessie-catalog': 'true', 'nessie.prefix-pattern': '{ref}|{warehouse}', 'nessie.default-branch.name': 'main'}
